![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with RedisVL

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this example we will load a list of advertisers with the following attributes: Advertiser Name, Category, Product Description, Target Demographics, and Region.

We will embed the advertiser product description so that users can search for advertisers that best match the type of product or audience they’re looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

## Packages

In [1]:
%pip install -q "redisvl>=0.6.0" sentence-transformers pandas nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import redis
redis.__version__

'6.4.0'

In [3]:
import redisvl
redisvl.__version__

'0.10.0'

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "redis-15951.c81.us-east-1-2.ec2.redns.redis-cloud.com") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "15951")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "training")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [5]:
from redis import Redis
client = Redis.from_url(REDIS_URL)
client.ping()

True

In [6]:
client.flushall()

True

### Load Advertisers Dataset

In [7]:
import pandas as pd
import numpy as np
import json

df = pd.read_json("resources/advertisers_real_100.json")
print("Loaded", len(df), "Advertiser entries")

df.head()

Loaded 50 Advertiser entries


,Advertiser Name,Category,Region,Product Description,Target Demographics,Location (HQ)
0,Nike,Fashion & Sportswear,North America,"Sports apparel, footwear, and accessories with...","18–40, athletes, fitness enthusiasts, sports fans","Beaverton, USA"
1,Adidas,Fashion & Sportswear,Europe,"Global sportswear brand offering footwear, app...","18–40, athletes, fitness and lifestyle consumers","Herzogenaurach, Germany"
2,Zara,Fashion & Apparel,Europe,Fast-fashion retailer with global reach,"16–35, Gen Z and young professionals","Arteixo, Spain"
3,H&M,Fashion & Apparel,Europe,"Affordable fashion retailer for men, women, an...","16–40, budget-conscious fashion buyers","Stockholm, Sweden"
4,Uniqlo,Fashion & Apparel,Asia,"Japanese casual wear designer, manufacturer, a...","18–45, minimalistic fashion enthusiasts","Tokyo, Japan"


In [8]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache

os.environ["TOKENIZERS_PARALLELISM"] = "false"


hf = HFTextVectorizer(
    model="sentence-transformers/all-MiniLM-L6-v2",
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=client,
    )
)

09:38:36 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
09:38:36 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [9]:
# Generate vectors by combining Product Description and Target Demographics
texts = (df["Target Demographics"].tolist())
df["vector"] = hf.embed_many(texts, as_buffer=True)

df.head()

,Advertiser Name,Category,Region,Product Description,Target Demographics,Location (HQ),vector
0,Nike,Fashion & Sportswear,North America,"Sports apparel, footwear, and accessories with...","18–40, athletes, fitness enthusiasts, sports fans","Beaverton, USA",b'\xb0\xacU=\xebZ+=G\xf51=;t\x11\xbc$^<=r\x17X...
1,Adidas,Fashion & Sportswear,Europe,"Global sportswear brand offering footwear, app...","18–40, athletes, fitness and lifestyle consumers","Herzogenaurach, Germany",b'n\x13q=s\xa3\x8e=\xe5\x90K\xbbg\xa0\xa3<z+\x...
2,Zara,Fashion & Apparel,Europe,Fast-fashion retailer with global reach,"16–35, Gen Z and young professionals","Arteixo, Spain",b'\xfa\xdc~;\xfd\xf6\xf5=7|M<\xd1\x7f^\xbc\xa2...
3,H&M,Fashion & Apparel,Europe,"Affordable fashion retailer for men, women, an...","16–40, budget-conscious fashion buyers","Stockholm, Sweden",b'D\t<=\trs=sN7<uz6=\x8e\x0e^=\x1c\x16\xd4\xbb...
4,Uniqlo,Fashion & Apparel,Asia,"Japanese casual wear designer, manufacturer, a...","18–45, minimalistic fashion enthusiasts","Tokyo, Japan",b'\xfa\x90\xdf<\x84\xfd8=`\x16\x81=\x80\xe5==....


## Define Redis index schema

In [10]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


index_name = "advertisers"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": index_name,
    "storage_type": "hash"
  },
  "fields": [
    {
        "name": "Advertiser Name",
        "type": "text",
    },
    {
        "name": "Product Description",
        "type": "text",
    },
    {
        "name": "Target Demographics",
        "type": "text",
    },
    {
        "name": "Category",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "Region",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
        {
        "name": "Location (HQ)",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [11]:
!rvl index info -i advertisers -u {REDIS_URL}



Index Information:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────╮
│ Index Name      │ Storage Type    │ Prefixes        │ Index Options   │ Indexing        │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
| advertisers     | HASH            | ['advertisers'] | []              | 0               |
╰─────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────╯
Index Fields:
╭─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────╮
│ Name                │ Attribute           │ Type                │ Field Option        │ Option Value        │ Field Option        │ Option Value        │ Field Option        │ Option Value        │ Field Option        │ Option Value        │
├────────────────

## Populate index

In [12]:
index.load(df.to_dict(orient="records"))

['advertisers:01K9FC399H7BBMBV84N0Z0JR62',
 'advertisers:01K9FC399H7BBMBV84N0Z0JR63',
 'advertisers:01K9FC399H7BBMBV84N0Z0JR64',
 'advertisers:01K9FC399H7BBMBV84N0Z0JR65',
 'advertisers:01K9FC399H7BBMBV84N0Z0JR66',
 'advertisers:01K9FC399H7BBMBV84N0Z0JR67',
 'advertisers:01K9FC399J7SZRZEE43MWJH3G6',
 'advertisers:01K9FC399J7SZRZEE43MWJH3G7',
 'advertisers:01K9FC399J7SZRZEE43MWJH3G8',
 'advertisers:01K9FC399J7SZRZEE43MWJH3G9',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GA',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GB',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GC',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GD',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GE',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GF',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GG',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GH',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GJ',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GK',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GM',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GN',
 'advertisers:01K9FC399J7SZRZEE43MWJH3GP',
 'advertise

## Search techniques

### Standard vector search

In [15]:
from redisvl.query import VectorQuery

user_query = "42 year male interested in country songs"

embedded_user_query = hf.embed(user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=10,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
)

result = index.query(vec_query)
pd.DataFrame(result)


,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399J7SZRZEE43MWJH3H1,0.590731263161,Spotify,Music streaming platform with global reach,Entertainment & Media,Europe,"Stockholm, Sweden"
1,advertisers:01K9FC399J7SZRZEE43MWJH3GZ,0.758590817451,Disney,"Entertainment giant with movies, streaming, an...",Entertainment & Media,North America,"Burbank, USA"
2,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.7666862607,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
3,advertisers:01K9FC399J7SZRZEE43MWJH3HB,0.768975317478,Marriott International,Hotel chain with global luxury and budget acco...,Travel & Leisure,Global,"Bethesda, USA"
4,advertisers:01K9FC399J7SZRZEE43MWJH3HC,0.768975317478,Hilton Hotels,Hospitality company with global hotels and res...,Travel & Leisure,Global,"McLean, USA"
5,advertisers:01K9FC399J7SZRZEE43MWJH3HD,0.772633552551,Amazon,"E-commerce platform, AWS, and consumer electro...",E-commerce & Retail,Global,"Seattle, USA"
6,advertisers:01K9FC399J7SZRZEE43MWJH3GT,0.780261397362,Citibank,"Banking, credit cards, and financial services",Finance,North America,"New York, USA"
7,advertisers:01K9FC399J7SZRZEE43MWJH3GG,0.783974409103,Sony,"Consumer electronics, PlayStation, and enterta...",Technology & Entertainment,Asia,"Tokyo, Japan"
8,advertisers:01K9FC399J7SZRZEE43MWJH3H9,0.785832047462,Booking.com,"Online travel agency offering hotels, flights,...",Travel & Leisure,Europe,"Amsterdam, Netherlands"
9,advertisers:01K9FC399J7SZRZEE43MWJH3GV,0.786301851273,HSBC,International banking and financial services,Finance,Europe/Asia,"London, UK"


### Vector search with filters

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

Search for top specifically in the Entertainment & Media:


In [16]:
from redisvl.query.filter import Tag

tag_filter = Tag("Category") == "Entertainment & Media"

vec_query.set_filter(tag_filter)

result=index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399J7SZRZEE43MWJH3H1,0.590731263161,Spotify,Music streaming platform with global reach,Entertainment & Media,Europe,"Stockholm, Sweden"
1,advertisers:01K9FC399J7SZRZEE43MWJH3GZ,0.758590817451,Disney,"Entertainment giant with movies, streaming, an...",Entertainment & Media,North America,"Burbank, USA"
2,advertisers:01K9FC399J7SZRZEE43MWJH3H0,0.802836418152,Warner Bros Discovery,"Movies, TV, HBO, streaming platforms",Entertainment & Media,North America,"New York, USA"
3,advertisers:01K9FC399J7SZRZEE43MWJH3GY,0.920409083366,Netflix,"Streaming platform offering movies, TV series,...",Entertainment & Media,North America,"Los Gatos, USA"


Search for top 10 advertisers specifically in the regios of North America and in Entertainment and Media.


In [17]:
from redisvl.query.filter import Num

# build combined filter expressions
tag_filter1 = Tag("Region") == "North America"
tag_filter2 = Tag("Category") == "Entertainment & Media"
combined_filter = tag_filter1 & tag_filter2

# build vector query
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=10,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
    filter_expression=combined_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399J7SZRZEE43MWJH3GZ,0.758590817451,Disney,"Entertainment giant with movies, streaming, an...",Entertainment & Media,North America,"Burbank, USA"
1,advertisers:01K9FC399J7SZRZEE43MWJH3H0,0.802836418152,Warner Bros Discovery,"Movies, TV, HBO, streaming platforms",Entertainment & Media,North America,"New York, USA"
2,advertisers:01K9FC399J7SZRZEE43MWJH3GY,0.920409083366,Netflix,"Streaming platform offering movies, TV series,...",Entertainment & Media,North America,"Los Gatos, USA"


Search with full text search for demographics that directly mention "fitness" in the description:


In [18]:
from redisvl.query.filter import Text

text_filter = Text("Target\ Demographics") % "fitness"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=10,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.7666862607,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
1,advertisers:01K9FC399H7BBMBV84N0Z0JR63,0.870938360691,Adidas,"Global sportswear brand offering footwear, app...",Fashion & Sportswear,Europe,"Herzogenaurach, Germany"


Vector search with wildcard text match:


In [19]:
text_filter = Text("Target\ Demographics") % "fit*"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.7666862607,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
1,advertisers:01K9FC399H7BBMBV84N0Z0JR63,0.870938360691,Adidas,"Global sportswear brand offering footwear, app...",Fashion & Sportswear,Europe,"Herzogenaurach, Germany"


Vector search with fuzzy match filter

> Note: fuzzy match is based on Levenshtein distance. Therefore, "fitness" might return result for "fit" as an example.

See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/


In [20]:

text_filter = Text("Target\ Demographics") % "%fitness%"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=10,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.7666862607,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
1,advertisers:01K9FC399H7BBMBV84N0Z0JR63,0.870938360691,Adidas,"Global sportswear brand offering footwear, app...",Fashion & Sportswear,Europe,"Herzogenaurach, Germany"


### Range queries

Range queries allow you to set a pre defined distance "threshold" for which we want to return documents. This is helpful when you only want documents with a certain "radius" from the search query.

In [21]:
from redisvl.query import RangeQuery

user_query = "15 year old highschool athlete rap music and podcasting"

embedded_user_query = hf.embed(user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

result = index.query(range_query)
pd.DataFrame(result)


,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399J7SZRZEE43MWJH3H1,0.4975233078,Spotify,Music streaming platform with global reach,Entertainment & Media,Europe,"Stockholm, Sweden"
1,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.640481948853,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
2,advertisers:01K9FC399H7BBMBV84N0Z0JR63,0.696381390095,Adidas,"Global sportswear brand offering footwear, app...",Fashion & Sportswear,Europe,"Herzogenaurach, Germany"
3,advertisers:01K9FC399J7SZRZEE43MWJH3G8,0.702062785625,Apple,"Consumer electronics, iPhone, Mac, iPad, and s...",Technology,North America,"Cupertino, USA"
4,advertisers:01K9FC399H7BBMBV84N0Z0JR64,0.715005218983,Zara,Fast-fashion retailer with global reach,Fashion & Apparel,Europe,"Arteixo, Spain"
5,advertisers:01K9FC399J7SZRZEE43MWJH3GM,0.721833765507,Starbucks,Coffeehouse chain offering beverages and snacks,Food & Beverage,North America,"Seattle, USA"
6,advertisers:01K9FC399J7SZRZEE43MWJH3GZ,0.733240783215,Disney,"Entertainment giant with movies, streaming, an...",Entertainment & Media,North America,"Burbank, USA"
7,advertisers:01K9FC399J7SZRZEE43MWJH3GY,0.741936922073,Netflix,"Streaming platform offering movies, TV series,...",Entertainment & Media,North America,"Los Gatos, USA"
8,advertisers:01K9FC399H7BBMBV84N0Z0JR66,0.757364749908,Uniqlo,"Japanese casual wear designer, manufacturer, a...",Fashion & Apparel,Asia,"Tokyo, Japan"
9,advertisers:01K9FC399J7SZRZEE43MWJH3GG,0.759241104126,Sony,"Consumer electronics, PlayStation, and enterta...",Technology & Entertainment,Asia,"Tokyo, Japan"


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `and` condition that returns vector search within the defined range and with a rating at or above 8.

In [26]:
range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
    distance_threshold=0.8
)

tag_filter = Tag("Region") == "North America"

range_query.set_filter(tag_filter)

# in this case we want to do a simple filter search or the vector so we execute as a joint filter directly
result = index.query(range_query)
pd.DataFrame(result)


,id,vector_distance,Advertiser Name,Product Description,Category,Region,Location (HQ)
0,advertisers:01K9FC399H7BBMBV84N0Z0JR62,0.640481948853,Nike,"Sports apparel, footwear, and accessories with...",Fashion & Sportswear,North America,"Beaverton, USA"
1,advertisers:01K9FC399J7SZRZEE43MWJH3G8,0.702062785625,Apple,"Consumer electronics, iPhone, Mac, iPad, and s...",Technology,North America,"Cupertino, USA"
2,advertisers:01K9FC399J7SZRZEE43MWJH3GM,0.721833765507,Starbucks,Coffeehouse chain offering beverages and snacks,Food & Beverage,North America,"Seattle, USA"
3,advertisers:01K9FC399J7SZRZEE43MWJH3GZ,0.733240783215,Disney,"Entertainment giant with movies, streaming, an...",Entertainment & Media,North America,"Burbank, USA"
4,advertisers:01K9FC399J7SZRZEE43MWJH3GY,0.741936922073,Netflix,"Streaming platform offering movies, TV series,...",Entertainment & Media,North America,"Los Gatos, USA"
5,advertisers:01K9FC399J7SZRZEE43MWJH3H0,0.775116324425,Warner Bros Discovery,"Movies, TV, HBO, streaming platforms",Entertainment & Media,North America,"New York, USA"


### Full text search

In [29]:
from redisvl.query import TextQuery

user_query = "fitness sports products"

text_query = TextQuery(
    text=user_query,
    text_field_name="Product\ Description",
    text_scorer="BM25STD",
    num_results=20,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
)

result = index.query(text_query)[:4]
print(result)
pd.DataFrame(result)[["Advertiser Name", "score"]]

[{'id': 'advertisers:01K9FC399H7BBMBV84N0Z0JR62', 'score': 9.306988485310125, 'Advertiser Name': 'Nike', 'Product Description': 'Sports apparel, footwear, and accessories with global brand appeal', 'Category': 'Fashion & Sportswear', 'Region': 'North America', 'Location (HQ)': 'Beaverton, USA'}, {'id': 'advertisers:01K9FC399J7SZRZEE43MWJH3H3', 'score': 6.245472511742946, 'Advertiser Name': 'Pfizer', 'Product Description': 'Pharmaceuticals, vaccines, and healthcare products', 'Category': 'Healthcare & Pharma', 'Region': 'North America', 'Location (HQ)': 'New York, USA'}, {'id': 'advertisers:01K9FC399J7SZRZEE43MWJH3H4', 'score': 5.951614013167226, 'Advertiser Name': 'Johnson & Johnson', 'Product Description': 'Healthcare, pharmaceuticals, consumer health products', 'Category': 'Healthcare & Pharma', 'Region': 'North America', 'Location (HQ)': 'New Brunswick, USA'}]


,Advertiser Name,score
0,Nike,9.306988
1,Pfizer,6.245473
2,Johnson & Johnson,5.951614


### Hybrid search

In [31]:
from redisvl.query import HybridQuery

hybrid_query = HybridQuery(
    text=user_query,
    text_field_name="Product\ Description",
    text_scorer="BM25",
    vector=embedded_user_query,
    vector_field_name="vector",
    alpha=0.9,
    num_results=10,
    return_fields=["Advertiser Name", "Product Description", "Category", "Region", "Location (HQ)"],
)

result = index.query(hybrid_query)[:4]
pd.DataFrame(result)[["Advertiser Name", "vector_similarity", "text_score", "hybrid_score"]]

,Advertiser Name,vector_similarity,text_score,hybrid_score
0,Nike,0.679759025574,2.11327127781,0.823110250798
1,Spotify,0.7512383461,0,0.67611451149
2,Adidas,0.651809304953,0,0.586628374457
3,Apple,0.648968607187,0,0.584071746469


### Next steps

For more query examples with redisvl: [see here](https://github.com/redis/redis-vl-python/blob/main/docs/user_guide/02_hybrid_queries.ipynb)

In [79]:
# clean up!
index.delete()